This is the Analysis Report for IMDb India Movies.
Thank you for providing excellent dataset!

# Loading Library and Dataset

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import warnings
warnings.filterwarnings("ignore")        
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.gridspec as grid_spec
import seaborn as sns

Dataset are loaded by pandas, and checked null via seaborn.

There are many null cells...

# Null Check

In [ ]:
df = pd.read_csv('/kaggle/input/imdb-india-movies/IMDb Movies India.csv')

plt.figure(figsize=(5,3),dpi=150)
sns.heatmap(df.isnull()).set_title('Null Check Graph')

Ok... need to check which column has most null data.

Around 50% of data is missing on 'Duration', 'Rating', 'Votes'

10%-20% of data missing, except 'Name'

In [ ]:
Div_by = df.shape[0] #rows = 15509

df.isnull().sum().sort_values(ascending=False)/Div_by

Droped data line when if there are null on 'Years', 'Genre'

It is not need for now

In [ ]:
df = df[df['Year'].notna()] 
df = df[df['Genre'].notna()] 

In [ ]:
df['Year'] = df['Year'].str.replace('(','').str.replace(')','').apply(pd.to_numeric)
df['Duration'] = df['Duration'].str.replace(' min','').apply(pd.to_numeric)

In [ ]:
df_refine = pd.concat([df, df['Genre'].str.get_dummies(sep=", ")], 1)

In [ ]:
df_refine.drop(['Name','Year','Duration','Genre','Rating','Votes','Director','Actor 1','Actor 2','Actor 3'], axis=1).sum().sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(7,3),dpi=150)
ax = sns.countplot(x="Year", data=df_refine)
ax.set_xticks(np.arange(2, 100, 10))
plt.show()

Graph looks nice, I can see 3 findings
1. Movie industry of India keep growing, all most 300-400 new movies created after mid 2010.
2. There was depression period on late 1990 and late 2000. I will find why.
3. What is movie year 2022? We are middle of 2021. Do some movies come from future?

Check again it is right dataset has year 2022.

In [ ]:
df[df['Year'] == 2022].sample(5)

Yes.. there are year 2022. It looks like the dataset has movies expected to be released in year 2022. It would not be problem, because Ratings and Votes are all null. 

In [ ]:
df_count = df_refine.drop(['Name','Duration','Genre','Rating','Votes','Director','Actor 1','Actor 2','Actor 3'], axis=1)
grouped = df_count.groupby('Year')
Genre_trend = grouped.sum()
Genre_trend_stacked = Genre_trend.stack()
Genre_trend_stacked_df = Genre_trend_stacked.reset_index(level=[0,1]).rename(columns = {'level_1': 'Genre', 0: 'Count'}, inplace = False)

Top_5 = ['Drama','Action','Romance','Comedy','Thriller']

Genre_trend_stacked_df = Genre_trend_stacked_df[Genre_trend_stacked_df['Genre'].isin(Top_5)]

In [ ]:
mask = Genre_trend_stacked_df['Year'].isin([2022])

plt.figure(figsize=(7,3),dpi=150)
sns.lineplot(data=Genre_trend_stacked_df[~mask], x="Year", y="Count", hue="Genre")

* Drama genre has been always top 1.
* Action genre was popular from 1980 to 2010, but after 2010, movie lovers in India did not like it as before.
* Trend of Thriller had maximum value on around 2005, but back to own trend.
* Comedy is now leading, top 2 after 2010.
* Romance has been stable.

In [ ]:
Actor1 = df[['Actor 1','Year']].rename(columns = {'Actor 1': 'Actor'}, inplace = False)
Actor2 = df[['Actor 2','Year']].rename(columns = {'Actor 2': 'Actor'}, inplace = False)
Actor3 = df[['Actor 3','Year']].rename(columns = {'Actor 3': 'Actor'}, inplace = False)
Actor_Info = pd.concat([Actor1, Actor2, Actor3], ignore_index=True).dropna()
Actor_Info['Count'] = 1

Test = Actor_Info['Actor'].value_counts().rename_axis('Actor').reset_index(name='Count')

Let's check who is the most apearance in India Movies.

In [ ]:
plt.figure(figsize=(4,3),dpi=150)
sns.barplot(x="Count", y="Actor", data=Test[0:10])

In [ ]:
Top = Test[0:10]['Actor']

Actor_Info = Actor_Info[Actor_Info['Actor'].isin(Top)]

Actor_Info = Actor_Info.set_index('Actor')
Actor_Info = Actor_Info.loc[Top]
Actor_Info = Actor_Info.reset_index(level=0)

And check their acting period

In [ ]:
plt.figure(figsize=(4,3),dpi=150)
sns.stripplot(x="Year", y="Actor",
              data=Actor_Info, dodge=True, alpha=.50, zorder=1)